# Агрегированные группировки датафреймов

Полная версия базы https://grouplens.org/datasets/movielens/ (265 М)

userId — идентификаторы пользователей, которые выставляли оценки фильмам;

movieId — идентификатор фильма.

rating — выставленный рейтинг фильма; 

timestamp — время выставления рейтинга. Это популярный формат даты и времени unixtime. Показывает количество секунд, прошедшее с 1 января 1970 года. Вы будете очень часто встречать этот формат.

In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
#Сколько уникальных значений userId в этом датафрейме

In [4]:
ratings['userId'].value_counts()

547    2391
564    1868
624    1735
15     1700
73     1610
452    1340
468    1291
380    1063
311    1019
30     1011
294     947
509     923
580     922
213     910
212     876
472     830
388     792
23      726
457     713
518     707
461     696
232     682
102     678
262     676
475     655
306     645
119     641
654     626
358     617
529     604
       ... 
356      21
579      21
319      20
14       20
448      20
583      20
76       20
310      20
1        20
498      20
438      20
638      20
651      20
325      20
399      20
289      20
209      20
296      20
445      20
337      20
249      20
540      20
604      20
668      20
657      20
221      20
444      20
484      20
35       20
485      20
Name: userId, Length: 671, dtype: int64

In [7]:
# Посчитаем количество уникальных значений user_id и минимальный рейтинг
print(len(ratings['userId'].unique()))

671


In [8]:
print(ratings['rating'].min())
print(ratings['rating'].max())

0.5
5.0


In [9]:
ratings.groupby('userId').count().head()

,movieId,rating,timestamp
userId,,,
1,20,20,20
2,76,76,76
3,51,51,51
4,204,204,204
5,100,100,100


In [10]:
# Для наглядности преобразуем эту таблицу в привычный датафрейм, оставив только нужные нам столбцы 
# (ID пользователя userId и количество строк в столбце movieId, т. е. в итоге количество оценок). 
# Выделить два и более столбцов можно с помощью двойных скобок:
ratings_count = ratings.groupby('userId').count().reset_index()[['userId', 'movieId']]
ratings_count.head()

,userId,movieId
0,1,20
1,2,76
2,3,51
3,4,204
4,5,100


In [11]:
ratings_count.sort_values('movieId').head()

,userId,movieId
0,1,20
497,498,20
447,448,20
444,445,20
443,444,20


Получите рейтинг пользователей, которые выставляли максимальное количество оценок фильмам. Каково максимальное количество оценок было выставлено одним пользователем?

In [12]:
ratings_count.sort_values('movieId', ascending=False).head()

,userId,movieId
546,547,2391
563,564,1868
623,624,1735
14,15,1700
72,73,1610


In [13]:
ratings.groupby('userId').count().max()

movieId      2391
rating       2391
timestamp    2391
dtype: int64

Теперь мы можем получить список пользователей, которые сделали 100 и более оценок. В итоге это будет просто питоновский список, и далее при расчете Lifetime мы отфильтруем всех пользователей по этому списку.

In [16]:
film_fans_ratings_count = ratings_count[ ratings_count['movieId'] >= 100 ]
film_fans_ratings_count.sort_values('movieId').head()

,userId,movieId
187,188,100
159,160,100
369,370,100
559,560,100
4,5,100


In [21]:
film_fans_user_ids = film_fans_ratings_count['userId'].tolist()
film_fans_user_ids[:5]

[4, 5, 8, 15, 17]

In [ ]:
#Получите аналогичный список для пользователей, которые поставили 50 и более оценок. 
#Выведите на экран первые 5 элементов получившегося списка.
#Какой ID имеет пятый пользователь в этом выводе? Ответ укажите как целое число

In [22]:
film_fans_ratings_count_50 = ratings_count[ ratings_count['movieId'] >= 50 ]
film_fans_user_ids_50 = film_fans_ratings_count_50['userId'].tolist()
film_fans_user_ids_50[:5]

[2, 3, 4, 5, 7]

ФИЛЬТРУЕМ РЕЙТИНГИ

Это все была подготовка к решению задачи. Напомню, мы хотим в огромном датафрейме с оценками посчитать рейтинг только по киноманам. Киноманов мы сформировали в лист film_fans_user_ids, теперь надо по выделенным нами пользователям вычленить данные из общей таблички. 

Осталось отфильтровать исходный датафрейм с рейтингами, оставив только пользователей из списка film_fans_user_ids. Для этого достаточно использовать метод isin.

Метод isin библиотеки Pandas для каждого значения столбца проверяет, входит ли это значение в список film_fans_user_ids. Этот метод удобно использовать в качестве фильтра для датафрейма.

В данном случае он оставит только те строки, у которых столбец userId содержится в списке film_fans_user_ids:

In [23]:
fans_data = ratings[ ratings['userId'].isin(film_fans_user_ids) ]
fans_data.head()

,userId,movieId,rating,timestamp
147,4,10,4.0,949810645
148,4,34,5.0,949919556
149,4,112,5.0,949810582
150,4,141,5.0,949919681
151,4,153,4.0,949811346


## ПРОВЕРОЧНОЕ ЗАДАНИЕ. ПОСЧИТАЕМ LIFETIME


Думали, мы все посчитаем за вас? Нет, теперь попробуйте со своими знаниями все сделать сами! Это очень просто, тем более шаги мы вам уже описали! Осталось только начать!

После получения датафрейма с киноманами можем посчитать искомую метрику Lifetime. В нашем случае  Lifetime пользователя — это разница между максимальным и минимальным значением столбца timestamp, т. е. средняя разница между последней и первой оценкой. Для расчета вам необходимо выполнить несколько шагов:

1. Сгруппировать датафрейм fans_data по userId и получить минимальное и максимально значение столбца timestamp. Напомним, что это можно получить в одну команду, используя метод agg(['min', 'max']).

2. В отдельном столбце рассчитать разницу 'diff' между минимальным и максимальным значением timestamp. После группировки у вас, скорее всего, будут столбцы со «вложенными» названиями. Для расчета разницы можно использовать следующий синтаксис (датафрейм после группировки и расчета min и max обозначен как min_max_df):

min_max_df['timestamp']['max'] - min_max_df['timestamp']['min']

3. Посчитать среднее значение столбца 'diff'. Это и будет значение Lifetime (в секундах).

4. Переведите это значение в дни.

In [28]:
min_max_df = fans_data.groupby('userId').agg(['min', 'max'])
# Считаем новый столбец diff
min_max_df['diff'] = min_max_df['timestamp']['max'] - min_max_df['timestamp']['min']
# Получаем ответ в секундах
min_max_df['diff'].mean() / 3600 / 24

455.2285713719898

In [48]:
min_max_df.head()

movieId         rating        timestamp                   diff
           min     max    min  max         min         max           
userId                                                               
4           10    4006    1.0  5.0   949778714   949982274     203560
5            3   48385    1.5  5.0  1163373044  1163375145       2101
8           32   44004    0.5  5.0  1154389340  1154474527      85187
15           1  161155    0.5  5.0   997937239  1469330735  471393496
17           6   34437    0.5  5.0  1127468587  1127476640       8053

# Сводные таблицы в Pandas

## ПОСТАНОВКА ЗАДАЧИ

В прошлых блоках мы группировали датафреймы по столбцам и получали набор метрик для них. Давайте расширим нашу задачу: требуется получить для каждого пользователя распределение по количеству выставленных оценок. Т. е. в строках таблицы указывается ID пользователя, а в столбцах — количество выставленных рейтингов: 0.5, 1.0...  5.0.

Для получения такого расширенного варианта группировок применяются сводные таблицы. Это те же сводные таблицы, которые широко используются в Excel.

ПРОСТЫЕ СВОДНЫЕ ТАБЛИЦЫ

# ДОМАШНЕЕ ЗАДАНИЕ
В прошлом шаге мы получили распределение оценок каждого фильма для значений от 0.5 до 5.0. Такая разбивка весьма детальна, а пользователю смотреть такие оценки неудобно. Ваша задача написать алгоритм, который классифицирует эти оценки по более практичной шкале:

оценка 2 и меньше — низкий рейтинг;
оценка 4 и меньше — средний рейтинг;
оценка 4.5 и 5 — высокий рейтинг.

Какую долю среди оценок фильма с movieId = 356 имеет 'средний рейтинг'? Ответ округлите до 2 знаков после запятой. Пример ответа: 0.32

In [47]:
data = pd.read_csv('ratings.csv')
data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [31]:
data[ data['userId'] == 1 ]['rating'].value_counts()

2.0    7
3.0    4
4.0    3
2.5    3
1.0    2
3.5    1
Name: rating, dtype: int64

In [33]:
data.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count')[0:10]

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
1,NaN,2.0,NaN,7.0,3.0,4.0,1.0,3.0,NaN,NaN
2,NaN,2.0,NaN,4.0,NaN,36.0,NaN,23.0,NaN,11.0
3,NaN,NaN,NaN,1.0,3.0,18.0,9.0,11.0,4.0,5.0
4,NaN,5.0,NaN,5.0,NaN,23.0,NaN,52.0,NaN,119.0
5,NaN,NaN,1.0,NaN,3.0,3.0,27.0,42.0,19.0,5.0
6,1.0,2.0,3.0,7.0,NaN,7.0,3.0,11.0,6.0,4.0
7,NaN,3.0,NaN,5.0,NaN,41.0,NaN,26.0,NaN,13.0
8,1.0,NaN,NaN,3.0,4.0,13.0,26.0,35.0,16.0,18.0
9,NaN,NaN,NaN,5.0,NaN,9.0,NaN,23.0,NaN,8.0


In [34]:
data_pivot = data.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0)
data_pivot.head()

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
1,0,2,0,7,3,4,1,3,0,0
2,0,2,0,4,0,36,0,23,0,11
3,0,0,0,1,3,18,9,11,4,5
4,0,5,0,5,0,23,0,52,0,119
5,0,0,1,0,3,3,27,42,19,5


In [35]:
data_pivot = data.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, margins=True)
data_pivot.head()

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,All
userId,,,,,,,,,,,
1,0,2,0,7,3,4,1,3,0,0,20
2,0,2,0,4,0,36,0,23,0,11,76
3,0,0,0,1,3,18,9,11,4,5,51
4,0,5,0,5,0,23,0,52,0,119,204
5,0,0,1,0,3,3,27,42,19,5,100


## Проверочное задание

(1 возможный балл)
Постройте рейтинг пользователей по количеству выставленных оценок 5.0.

Какое максимальное количество 'пятерок' выставил один пользователь?

In [36]:
# Найдем, какое максимальное количество пятерок поставил один пользователей
data_pivot.sort_values(5.0, ascending=False) # 408 - искомый ответ

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,All
userId,,,,,,,,,,,
All,1101,3326,1687,7271,4449,20064,10538,28750,7723,15095,100004
564,0,152,0,187,0,414,0,707,0,408,1868
232,0,35,0,32,0,96,0,276,0,243,682
242,0,0,0,4,0,25,0,151,0,219,399
547,53,79,58,204,177,411,378,591,226,214,2391
30,0,20,0,86,0,201,2,505,1,196,1011
472,0,23,1,56,5,188,13,315,40,189,830
102,0,8,0,40,0,87,0,369,0,174,678
358,0,114,0,91,0,118,0,156,0,138,617


# Агрегированные столбцы датафреймов

## ПОСТАНОВКА ЗАДАЧИ

В этом блоке мы разберем пример расчета рейтингов фильма, т. е. долей оценок внутри одного фильма. В итоге нам необходимо написать алгоритм, показывающий распределение оценок для заданного фильма movieId: какая доля оценок пришлась на 0.5, на 1.0... на 5.0.

При этом для расчета доли оценки нам надо знать сумму всех оценок каждого фильма. В разных системах такой класс задач называется оконными, кумулятивными функциями, партиционированием. Также они позволяют подсчитывать рейтинги внутри групп, бегущую сумму и другие полезные для аналитика метрики (специальные методы под эти задачи есть в Pandas). Мы рассмотрим один из них с использованием метода transform.

In [41]:
data.pivot_table(index = ['movieId', 'rating'], values = 'timestamp', aggfunc = 'count').head()

timestamp
movieId rating           
1       1.0             4
        1.5             3
        2.0            13
        2.5             4
        3.0            41

In [42]:
# Разворачиваем Пивот в таблицу
data_pivot = data.pivot_table(index = ['movieId', 'rating'], values = 'timestamp', aggfunc = 'count').reset_index()
data_pivot.head(10)

,movieId,rating,timestamp
0,1,1.0,4
1,1,1.5,3
2,1,2.0,13
3,1,2.5,4
4,1,3.0,41
5,1,3.5,23
6,1,4.0,77
7,1,4.5,19
8,1,5.0,63
9,2,1.5,1


In [43]:
data_pivot['sum'] = data_pivot.groupby('movieId').timestamp.transform(lambda x: sum(x))
data_pivot.head(10)

,movieId,rating,timestamp,sum
0,1,1.0,4,247
1,1,1.5,3,247
2,1,2.0,13,247
3,1,2.5,4,247
4,1,3.0,41,247
5,1,3.5,23,247
6,1,4.0,77,247
7,1,4.5,19,247
8,1,5.0,63,247
9,2,1.5,1,107


In [45]:
data_pivot['share'] = data_pivot['timestamp'] / data_pivot['sum']
data_pivot.head(10)

,movieId,rating,timestamp,sum,share
0,1,1.0,4,247,0.016194
1,1,1.5,3,247,0.012146
2,1,2.0,13,247,0.052632
3,1,2.5,4,247,0.016194
4,1,3.0,41,247,0.165992
5,1,3.5,23,247,0.093117
6,1,4.0,77,247,0.311741
7,1,4.5,19,247,0.076923
8,1,5.0,63,247,0.255061
9,2,1.5,1,107,0.009346


In [46]:
# Найдем сумму столбца share для movieId = 1
print(data_pivot[data_pivot['movieId'] == 1]['share'].sum())

1.0


## ДОМАШНЕЕ ЗАДАНИЕ

В прошлом шаге мы получили распределение оценок каждого фильма для значений от 0.5 до 5.0. Такая разбивка весьма детальна, а пользователю смотреть такие оценки неудобно. Ваша задача написать алгоритм, который классифицирует эти оценки по более практичной шкале:

оценка 2 и меньше — низкий рейтинг;
оценка 4 и меньше — средний рейтинг;
оценка 4.5 и 5 — высокий рейтинг.

Какую долю среди оценок фильма с movieId = 356 имеет 'средний рейтинг'? Ответ округлите до 2 знаков после запятой. Пример ответа: 0.32

In [50]:
data_pivot['class'] = data_pivot['rating'].apply(lambda x: 'Низкий' if x <= 2 else ('Высокий' if x > 4 else 'Средний'))
data_pivot.head(10)

,movieId,rating,timestamp,sum,share,class
0,1,1.0,4,247,0.016194,Низкий
1,1,1.5,3,247,0.012146,Низкий
2,1,2.0,13,247,0.052632,Низкий
3,1,2.5,4,247,0.016194,Средний
4,1,3.0,41,247,0.165992,Средний
5,1,3.5,23,247,0.093117,Средний
6,1,4.0,77,247,0.311741,Средний
7,1,4.5,19,247,0.076923,Высокий
8,1,5.0,63,247,0.255061,Высокий
9,2,1.5,1,107,0.009346,Низкий


In [51]:
print(data_pivot[(data_pivot['movieId'] == 356) & (data_pivot['class'] == 'Средний')]['share'].sum())

0.5513196480938416
